# University Student Dropout: A Longitudinal Dataset of Demographic, Socioeconomic, and Academic Indicators
In this document I walk through the data provided by this research and do basic training before separating code into final executables


In [ ]:
import pandas as pd

In [ ]:
df_2018 = pd.read_csv("data/raw/dataset_2018_hash.csv")

: 